## Evalute the entanglement of real-valued quantum states

Negativity is a measure of quantum entanglement which is easy to compute if one already obtains the density matrix $\rho$. It is a measure deriving from the PPT criterion for separability [1]. Directly compute negativity on quantum hardware is challenging since the partial transpose operation involved is non-physical, which could generate non-postive eigenvalues and hence not a CPTP map. 

The logarithmic negativity is defined as
$$
E_{\text{N}}(\rho) \equiv \log _2\left\|\rho^{\Gamma_A}\right\|_1,
\tag{1}
$$
where $\rho^{\Gamma_A}$ is the partial transpose of $\rho$ with respect to subsystem $A$ and $\|X\|_1=\operatorname{Tr}|X|=\operatorname{Tr} \sqrt{X^{\dagger} X}$ is the trace norm or the sum of the singular values of the operator  $X$.


### Benchmark the entanglement meausre

It is known that $E_{\text{N}}(\rho)=0$ for separable states $\rho = \rho_A \otimes \rho_B$. 

In [3]:
import numpy as np
import hadamard_random_forest as hrf

num_sysA = 2
num_sysB = 3
num_qubits = num_sysA + num_sysB
psi1 = hrf.random_statevector(num_sysA)
psi2 = hrf.random_statevector(num_sysB)
sv = np.kron(psi1, psi2).reshape(2**(num_qubits),1)
rho = sv @ sv.T.conj()

LN = hrf.logarithmic_negativity(rho, num_sysA, num_qubits)
print("Logarithmic Negativity (exact) =", LN)

Logarithmic Negativity (exact) = 4.4847971053408774e-15


For the Bell state $\left|\Phi^{+}\right\rangle = (|00\rangle+|11\rangle)/\sqrt{2}$, one can verify $E_{\text{N}}(\left|\Phi^{+}\right\rangle\langle \Phi^{+}|)=1$.

In [4]:
import numpy as np
import hadamard_random_forest as hrf

num_sysA = 1
num_sysB = 1
num_qubits = num_sysA + num_sysB
sv = np.array([[1,0,0,1]]/np.sqrt(2)).reshape([4,1])
rho = sv @ sv.T.conj()

LN = hrf.logarithmic_negativity(rho, num_sysA, num_qubits)
print("Logarithmic Negativity (exact) =", LN)

Logarithmic Negativity (exact) = 0.9999999999999997


### Use HRF to estimate negativity

## Reference

[1] [Życzkowski, Karol, et al. "Volume of the set of separable states." Physical Review A 58.2 (1998): 883.](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.58.883)

---

##